In [357]:
%%capture
%run 02_transaction.ipynb
%run 03_merkle.ipynb
%run 04_account.ipynb 

In [358]:
from objsize import get_deep_size as get_size

## Block Header

The `Header` is a blocks summary. It only stores the `root` node of the txs merkle tree, which makes it small and constant in size. The actual mining happens on the block header.

In [359]:
class Header(Hashable):
    def __init__(self, root, prev_hash, number, n_txs):
        self.root      = root
        self.prev_hash = prev_hash
        self.number    = number
        self.n_txs     = n_txs
        self.mined     = False
        self.time      = time.time()
        
    def __setattr__(self, prop, val):
        super().__setattr__(prop, val) 
        if prop != 'nonce': super().__setattr__('hash', sha(self.__dict__))
            
    def __str__(self):  return stringfy(self)

In [360]:
bh = Header(rh(),rh(),2,10); print(bh)

root:          📒 0x6cad62a4c0...9de
hash:          🔛 0xb5ccbe209e...c3f
prev_hash:     📾 0x98a9db0dc0...b27
number:        2
n_txs:         10
mined:         False
time:          Wed Mar 31 10:12:08 2021


If anything in the block header changes its `hash` changes automatically as well.

In [361]:
bh_changed        = deepcopy(tx2)
bh_changed.number = 120
assert bh != bh_changed

### Info

Includes all extra informations that are irrelevant for mining and therefore are not in the block header.

Calculate the total txs `volume` and `fees` included in a block.

In [362]:
def stats(txs):
    volume = sum([tx.value for tx in txs])
    fees   = sum([tx.fee   for tx in txs])
    return volume, fees

`Info` calculates some interesting facts about the `txs` that will be included in the block.

In [363]:
class Info:
    def __init__(self, txs):
        self.volume,self.fees = stats(txs)
        self.sz = get_size(txs)
    def __str__(self): return stringfy(self)

## Block

Consists of the block header `bh` and a list of txs. To validate that the txs are correct, the merkle tree can be rebuild and checked against the root hash stored in the `bh`.

In [369]:
class Block(Hashable): 
    def __init__(self, bh, txs):
        self.info   = Info(txs)
        self.header = bh
        self.hash   = bh.hash
        self.mt     = MerkleTree(txs)
        self.txs    = self.val_txs(txs)
        
    def val_txs(self, txs):
        for tx in txs: assert val_sig(tx),       'tx signature invalid'
        assert self.are_unique(txs),             'txs include duplicates'
        assert self.mt.root == self.header.root, 'txs root hash do not match'
        return {tx.hash: tx for tx in txs}
    
    def are_unique(self, txs): return len(set([tx.hash for tx in txs])) == len(txs)
    def __getitem__(self, key):return self.txs[key] 
    def __str__(self):         return (str(self.header)
                                       +'\n'+str(self.info)
                                       +'\n\ntxs:\n' 
                                       +'\n'.join(tx.smry() for tx in self.txs.values()))

Create random list of signed txs.

In [380]:
acc1,acc2 = Account(),Account()
def r_stxs(n): return [acc1.signed_tx(acc2,ri(1,9),ri(1,9)/100) for _ in range(n)]

Create block from a block header containing its txs merkle tree.

In [381]:
txs = r_stxs(4)
mt  = MerkleTree(txs)
bh  = Header(mt.root, rh(), 2, len(txs))
b   = Block(bh, txs); print(b)

root:          📇 0x61810bbd17...d8b
hash:          👫 0x0532e990b6...f3c
prev_hash:     💓 0x2da53839a5...f11
number:        2
n_txs:         4
mined:         False
time:          Wed Mar 31 10:16:01 2021
volume:        28.0 eth
fees:          0.27 eth
sz:            4458

txs:
👶 0x1076 -> 📘 0x72f9 6.0 eth
👶 0x1076 -> 📘 0x72f9 7.0 eth
👶 0x1076 -> 📘 0x72f9 9.0 eth
👶 0x1076 -> 📘 0x72f9 6.0 eth


Access specific tx from block with its hash.

In [382]:
tx_hash = next(iter(b.txs.keys())); tx_hash

'0x7a5b7ca3fe2ac38248965974550607b96670f61ad9c27de1ecb14e32dd5a3d4f'

In [383]:
tx = b[tx_hash]; print(tx)

fr:            👶 0x107679890F...50F
hash:          📠 0x7a5b7ca3fe...d4f
to:            📘 0x72f925dbe8...e4c
value:         6.0 eth
fee:           0.05 eth
nonce:         0
time:          Wed Mar 31 10:16:01 2021
signed:        True
sig:           🕢 0xfc8230c3db...453


Blocks can only contain unique txs. 

In [384]:
txs[0] = txs[1]

mt  = MerkleTree(txs)
bh  = Header(mt.root, rh(), 2, len(txs))

In [385]:
dup_detected = False
try:    Block(bh, txs)
except: dup_detected = True
assert  dup_detected